In [52]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [53]:
transform = transforms.ToTensor() 
# 이미지를 pytorch tensor 로 변환
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data, batch_size=64, shuffle=True) # 60000장의 data를 64 장씩 random 으로 뽑아서 총 937개가 있는 data -> data와 label 로 구성되어 있음

In [54]:
dataiter = iter(data_loader)
images, labels = dataiter.next() # batch size가 64인 data [64,1,28,28]
print(images.shape) # torch.Size([64, 1, 28, 28])
# 1개 image의 pixel값이 어떤 범위에 있는지 ,# output 출력할 때 활성화 함수 정하기 위해 필요
print(torch.min(images), torch.max(images)) 

torch.Size([64, 1, 28, 28])
tensor(0.) tensor(1.)


In [55]:
# nn.Sequential 짤 때, 콤마(,) 점(.) 으로 찍는 것 조심
class Autoencoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder=nn.Sequential(
        nn.Linear(28*28, 250),
        nn.ReLU(),
        nn.Linear(250,50),
        nn.ReLU(),
        nn.Linear(50,10)  # (N,10)
    )

    self.decoder=nn.Sequential(
        nn.Linear(10,50),
        nn.ReLU(),
        nn.Linear(50,250),
        nn.ReLU(),
        nn.Linear(250,784),
        nn.Sigmoid()  # 입력이 0-1 사이 이므로 출력도 0-1 사이가 되도록
    )

  # 복원된 image 구하기
  def forward(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded # encoded 로 바꾸면 latent vector 출력 가능

  # embedding -> latent vector 구하기
  def encoding(self,x):
    encoded = self.encoder(x)
    return encoded

In [56]:
sample_data=images[0].view(-1,784)
print(sample_data)
model = Autoencoder()
print(model(sample_data)) # 생성자에 바로 data 붙어서 forward 함수 사용 가능한 듯, 재구성된 image
encoded_data=model.encoding(sample_data) # encode된 data
print(encoded_data.shape) # torch.Size([1, 10])
print(encoded_data) # tensor([[-0.1001,  0.0028, -0.0790, -0.1277,  0.1153,  0.0227,  0.0345,  0.1823, 0.0391, -0.0742]], grad_fn=<AddmmBackward0>)

print(torch.min(encoded_data), torch.max(encoded_data))


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [57]:
# heuristic 한 방식으로 label 값 찾기
print(mnist_data[1][1]) # 0
print(mnist_data[3][1]) # 1
print(mnist_data[5][1]) # 2
print(mnist_data[7][1]) # 3
print(mnist_data[2][1]) # 4
print(mnist_data[0][1]) # 5
print(mnist_data[13][1]) # 6
print(mnist_data[15][1]) # 7
print(mnist_data[17][1]) # 8
print(mnist_data[4][1]) # 9

0
1
2
3
4
5
6
7
8
9


In [58]:
# heuristic 한 방식으로 초기 중심점 잡기
c_0 = model.encoder(mnist_data[1][0].view(-1,28*28)) #0
c_1 = model.encoder(mnist_data[3][0].view(-1,28*28)) #1
c_2 = model.encoder(mnist_data[5][0].view(-1,28*28)) #2
c_3 = model.encoder(mnist_data[7][0].view(-1,28*28)) #3
c_4 = model.encoder(mnist_data[2][0].view(-1,28*28)) #4
c_5 = model.encoder(mnist_data[0][0].view(-1,28*28)) #5
c_6 = model.encoder(mnist_data[13][0].view(-1,28*28)) #6
c_7 = model.encoder(mnist_data[15][0].view(-1,28*28)) #7
c_8 = model.encoder(mnist_data[17][0].view(-1,28*28)) #8
c_9 = model.encoder(mnist_data[4][0].view(-1,28*28)) #9

print(c_0)

tensor([[ 0.0973, -0.1241, -0.0425, -0.0113, -0.0103, -0.0838, -0.0018,  0.1475,
         -0.0233, -0.1897]], grad_fn=<AddmmBackward0>)


In [59]:
# 초기 중심값 선정방법 1 -> 0-9 까지의 label data 를 encoding 시킨 것 
init_centroids={}
for i in range(10):
  init_centroids['C{}'.format(i)]=0 # 딕셔너리 key 값 동적생성 
init_centroids['C0']=c_0
init_centroids['C1']=c_1
init_centroids['C2']=c_2
init_centroids['C3']=c_3
init_centroids['C4']=c_4
init_centroids['C5']=c_5
init_centroids['C6']=c_6
init_centroids['C7']=c_7
init_centroids['C8']=c_8
init_centroids['C9']=c_9
print(init_centroids)
# 초기 중심값 선정방법 2 -> 그냥 random 으로 중심점 선택

{'C0': tensor([[ 0.0973, -0.1241, -0.0425, -0.0113, -0.0103, -0.0838, -0.0018,  0.1475,
         -0.0233, -0.1897]], grad_fn=<AddmmBackward0>), 'C1': tensor([[ 0.1066, -0.0990, -0.0939, -0.0101, -0.0756, -0.0836,  0.0442,  0.1266,
         -0.0264, -0.1478]], grad_fn=<AddmmBackward0>), 'C2': tensor([[ 0.1136, -0.0893, -0.0766, -0.0171, -0.0171, -0.1161,  0.0512,  0.1273,
         -0.0168, -0.1789]], grad_fn=<AddmmBackward0>), 'C3': tensor([[ 0.1303, -0.1125, -0.0973,  0.0105, -0.0490, -0.1170,  0.0559,  0.1619,
         -0.0141, -0.1999]], grad_fn=<AddmmBackward0>), 'C4': tensor([[ 0.0809, -0.0752, -0.1053, -0.0038, -0.0262, -0.1022,  0.0289,  0.1278,
         -0.0198, -0.1569]], grad_fn=<AddmmBackward0>), 'C5': tensor([[ 0.1365, -0.1344, -0.0478, -0.0386, -0.0336, -0.1404,  0.0142,  0.1289,
         -0.0189, -0.1302]], grad_fn=<AddmmBackward0>), 'C6': tensor([[ 0.0687, -0.0687, -0.0870, -0.0043, -0.0288, -0.0784,  0.0164,  0.1331,
         -0.0408, -0.1658]], grad_fn=<AddmmBackward0>)

In [60]:
# 가장 가까운 key 값 return 하는 함수
# dictinary에 'C0' , 'C1' ... 이 KEY로 들어가고 value 가 tensor 로 만들고 정렬해서 최소값에 해당하는 key 뽑기
def closest_center(centroids, encoded_x):
  distances={}
  for key in centroids.keys():
    distance=torch.sum((centroids[key] - encoded_x)**2).item()
    distances[key]=distance
  # print(distances) # {'C0': 0.0, 'C1': 0.012900883331894875, 'C2': 0.006580686662346125 .... }
  return sorted(distances.items(), key=lambda x:x[1])[0][0]

print(closest_center(init_centroids, model.encoding(mnist_data[1][0].view(-1,28*28)))) # 'str' type

C0


In [40]:
# 할당 dictionary 초기화
# label 당 개수로 확인하면 좀 더 빠르게 확인 가능
# 차라리 index를 저장하는 방향으로 하면 더 나을것 같기도
init_assignments={}
for i in range(10):
  init_assignments['C{}'.format(i)]={} # 딕셔너리 key 값 동적생성 , value는 딕셔너리 형태로 들어감

for key in init_assignments.keys():
  for i in range(10):
    init_assignments[key][i]=0

print(init_assignments)

# 초기 x값들을 초기 중심에 할당 , 60000 개 학습하는데 30초
for img,label in mnist_data:
  new_img = img.view(-1,28*28)
  encoded_img=model.encoding(new_img)
  key = closest_center(init_centroids, encoded_img)

  dict = init_assignments[key] # 해당 key에 맞는 딕셔너리 불러옴
  dict[label]=dict[label]+1



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'C0': 0.010916374623775482, 'C1': 0.0012652060249820352, 'C2': 0.006486483383923769, 'C3': 0.006193498615175486, 'C4': 0.0027850621845573187, 'C5': 0.00871329102665186, 'C6': 0.004267488140612841, 'C7': 0.005019532982259989, 'C8': 0.0025651955511420965, 'C9': 0.0057196165435016155}
{'C0': 0.022094106301665306, 'C1': 0.0030275846365839243, 'C2': 0.010891823098063469, 'C3': 0.015856826677918434, 'C4': 0.005332987755537033, 'C5': 0.016194412484765053, 'C6': 0.010063969530165195, 'C7': 0.014577530324459076, 'C8': 0.008046496659517288, 'C9': 0.014775129035115242}
{'C0': 0.00848146341741085, 'C1': 0.009053134359419346, 'C2': 0.0037680843379348516, 'C3': 0.008898302912712097, 'C4': 0.011289210990071297, 'C5': 0.003394289640709758, 'C6': 0.011398609727621078, 'C7': 0.008099307306110859, 'C8': 0.004669464658945799, 'C9': 0.019564030691981316}
{'C0': 0.007309312466531992, 'C1': 0.014091691933572292, 'C2': 0.01007944904267788, 'C3': 0.009032262489199638, 'C4':

In [41]:
print(init_assignments)

{'C0': {0: 1230, 1: 1, 2: 108, 3: 151, 4: 4, 5: 225, 6: 5, 7: 5, 8: 22, 9: 8}, 'C1': {0: 294, 1: 1925, 2: 926, 3: 157, 4: 1200, 5: 347, 6: 208, 7: 565, 8: 469, 9: 1017}, 'C2': {0: 1463, 1: 84, 2: 2422, 3: 2251, 4: 693, 5: 1292, 6: 4139, 7: 431, 8: 1665, 9: 661}, 'C3': {0: 111, 1: 2, 2: 139, 3: 554, 4: 423, 5: 80, 6: 9, 7: 128, 8: 94, 9: 220}, 'C4': {0: 112, 1: 2555, 2: 457, 3: 363, 4: 557, 5: 246, 6: 161, 7: 405, 8: 287, 9: 367}, 'C5': {0: 1165, 1: 11, 2: 258, 3: 496, 4: 13, 5: 671, 6: 362, 7: 275, 8: 360, 9: 80}, 'C6': {0: 254, 1: 5, 2: 194, 3: 141, 4: 286, 5: 277, 6: 283, 7: 91, 8: 22, 9: 94}, 'C7': {0: 421, 1: 285, 2: 108, 3: 571, 4: 1075, 5: 744, 6: 51, 7: 2419, 8: 330, 9: 1614}, 'C8': {0: 872, 1: 1720, 2: 1345, 3: 1439, 4: 1155, 5: 1497, 6: 697, 7: 1337, 8: 2579, 9: 1418}, 'C9': {0: 1, 1: 154, 2: 1, 3: 8, 4: 436, 5: 42, 6: 3, 7: 609, 8: 23, 9: 470}}


In [61]:
# 할당 dictionary 초기화
# label 당 개수로 확인하면 좀 더 빠르게 확인 가능
init_assignments={}
for i in range(10):
  init_assignments['C{}'.format(i)]={} # 딕셔너리 key 값 동적생성 , value는 딕셔너리 형태로 들어감

for key in init_assignments.keys():
  for i in range(10):
    init_assignments[key][i]=[]

print(init_assignments)

# 초기 x값들을 초기 중심에 할당 , 60000 개 학습하는데 30초
for img,label in mnist_data:
  new_img = img.view(-1,28*28)
  encoded_img=model.encoding(new_img)
  key = closest_center(init_centroids, encoded_img)

  dict = init_assignments[key] # 해당 key에 맞는 딕셔너리 불러옴
  dict[label].append(new_img)


{'C0': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C1': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C2': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C3': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C4': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C5': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C6': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C7': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C8': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}, 'C9': {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}}


In [69]:
# 너무 data 많아서 오류가 나는 듯함 -> 해결 필요
print(init_assignments)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
a=torch.tensor([[0.0,0.0,0.0,0.0]])
b=torch.tensor([[1.0,1.0,1.0,1.0]])
print((a+b)/2)

tensor([[0.5000, 0.5000, 0.5000, 0.5000]])


In [68]:
# 중심 update 하는 함수
def update_centroid(cluster_assignments, model):
  for key in cluster_assignments.keys():
    dict={}
    sum = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
    len=0
    for i in cluster_assignments[key].keys():
      for x in cluster_assignments[key][i]:
        sum = sum + model.encoding(x)
        len = len + 1
    dict[key]= sum / len
  return dict

print(update_centroid(init_assignments,model))

{'C9': tensor([[  0.1625, -17.6565,  10.4377,   4.6056,   2.3904,  -2.1616,  -4.6705,
           3.4581,   5.3045,   5.8963]], grad_fn=<DivBackward0>)}


In [64]:
# 새로운 중심에 x 들을 할당하는 함수
def assign_cluster(cluster_centroids, model):
  cluster_assignments={}
  for i in range(10):
    cluster_assignments['C{}'.format(i)]={} # 딕셔너리 key 값 동적생성 , value는 딕셔너리 형태로 들어감

  for key in cluster_assignments.keys():
    for i in range(10):
      cluster_assignments[key][i]=[]

  for img,label in mnist_data:
    new_img = img.view(-1,28*28)
    encoded_data = model.encoding(new_img)
    key = closest_center(cluster_centroids, encoded_data)
    dict = cluster_assignments[key]
    dict[label].append(new_img)
  
  return cluster_assignments

In [65]:
# cluster loss update 하는 함수
def update_cluster_loss(cluster_centroids, cluster_assignments, model):
  cluster_loss=0.0
  for key in cluster_assignments.keys():
    centroid = cluster_centroids[key]
    for i in cluster_assignments[key].keys():
      for x in cluster_assignments[key][i]:
        encoded_data = model.encoding(x)
        cluster_loss = cluster_loss + torch.sum((centroid-encoded_data)**2).item()
        
  return cluster_loss
  

In [35]:
a=torch.tensor([[1.0,2.0,3.0]])
a=torch.sum(a)
print(a.item()) # 값으로 변환가능

6.0


In [66]:
 # Training 과정 -> 함수화
 # loss = reconstruct_error + cluster_loss 로 일단 함
 # 1. 1epoch 동안 신경망의 매개변수 update
 # 2. 전체 훈련 data 60000 장에 대해서 clustering 진행 -> 중심 좌표들을 update
 # 3. 전체 훈련 data 에 대해서 가장 가까운 중심에 할당시킴
 # 4. 1,2,3 과정을 T (hyperparameter) 동안 반복

# autoencoder 객체 , Loss 종류, Optimizer 종류, lambda
def train(model, criterion, optimizer, T, cluster_loss, cluster_centroids, cluster_assignments):
    for _ in range(T): # T번 반복
        # 1. 1epoch 동안 신경망의 매개변수 update
        for img,labels in data_loader:
            img = img.view(-1,28*28) # torch.Size([64, 784])
            recon = model.forward(img)
            loss = criterion(img,recon) + cluster_loss

            optimizer.zero_grad() # 배치마다 gradient 를 0으로 초기화
            loss.backward()
            optimizer.step()
      
        # 2. 전체 훈련 data 60000 장에 대해서 clustering 진행 -> 중심 좌표들을 update
        cluster_centroids = update_centroid(cluster_assignments, model) 

        # 3. update 된 중심 좌표에 x 값들 할당
        cluster_assignments = assign_cluster(cluster_centroids, model)

        # 4. clustering loss update
        cluster_loss = update_cluster_loss(cluster_centroids, cluster_assignments, model)
      
    return cluster_assignments


In [67]:
criterion = nn.MSELoss() # Mean Squared Error 가 Loss function이 되도록 함 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5) # parameter를 갱신하는 것을 optimizer라고 함
assign = train(model,criterion, optimizer, 3, 0, init_centroids, init_assignments)


KeyError: ignored